# DATASET SISANT - Cadastro de Drones

### Este dataset contém os registros de drones até o dia 26/10/2022.
#### O dataset possui informações cadastrais, então as observações serão voltadas a clusterização e medição.

In [1]:
#Bibliotecas
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
#DataFrame: a primeira linha do arquivo informa a data de extração do relatório
df = pd.read_csv('SISANT.csv',skiprows=0 ,header=1, sep=';')

In [3]:
#Verifica o tamanho do DataFrame
df.size

1038120

In [4]:
#Exibe os tipos de dados do DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103812 entries, 0 to 103811
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   CODIGO_AERONAVE           103812 non-null  object 
 1   DATA_VALIDADE             103812 non-null  object 
 2   OPERADOR                  103812 non-null  object 
 3   CPF_CNPJ                  103812 non-null  object 
 4   TIPO_USO                  103812 non-null  object 
 5   FABRICANTE                103808 non-null  object 
 6   MODELO                    103808 non-null  object 
 7   NUMERO_SERIE              103681 non-null  object 
 8   PESO_MAXIMO_DECOLAGEM_KG  103812 non-null  float64
 9   RAMO_ATIVIDADE            103772 non-null  object 
dtypes: float64(1), object(9)
memory usage: 7.9+ MB


In [5]:
#Verifica duplicidades
df.duplicated().value_counts()

False    103796
True         16
dtype: int64

As duplicidades devem ser excluídas, pois são uma anomalia nesse dataset.

In [6]:
#Apaga as duplicidades
df.drop_duplicates(inplace=True)

In [7]:
#Apaga as colunas que não serão analisadas
del df['OPERADOR']
del df['CPF_CNPJ']
del df['NUMERO_SERIE']

In [8]:
#Exibe as primeiras linhas do DataFrame
df.head()

,CODIGO_AERONAVE,DATA_VALIDADE,TIPO_USO,FABRICANTE,MODELO,PESO_MAXIMO_DECOLAGEM_KG,RAMO_ATIVIDADE
0,PP- 11082012,05/01/2024,Básico,DJI,Mavic,0.743,Aerocinematografia
1,PP- 26534803,07/04/2024,Básico,DJI,Mavic 2 Pro,9.000,Aerofotografia
2,PP- 35041299,03/01/2024,Básico,DJI,Mavic,0.128,Aeroinspeção
3,PP- 40211550,05/05/2023,Básico,DJI,Phantom 4 Pro,1.388,Aerolevantamento - Aerofotogrametria
4,PP-000000001,26/03/2023,Básico,DJI,Mavic,1.000,Aerocinematografia


In [9]:
#Verifica registros nulos
df.isnull().sum()

CODIGO_AERONAVE              0
DATA_VALIDADE                0
TIPO_USO                     0
FABRICANTE                   4
MODELO                       4
PESO_MAXIMO_DECOLAGEM_KG     0
RAMO_ATIVIDADE              40
dtype: int64

Não será aplicado qualquer tratamento nos valores nulos, pois as principais colunas para a análise possuem de 0 à 4 valores nulos e não vão interferir na análise.

### Exploração do dataset

In [10]:
#Verificação sobre a data de validade
print('Validade Mínima:',df.DATA_VALIDADE.min())
print('Validade Máxima:',df.DATA_VALIDADE.max())

Validade Mínima: 01/01/2023
Validade Máxima: 31/12/2023


Todos os registros do DataFrame estão dentro da validade.

In [11]:
#Verificação sobre os fabricantes
df.FABRICANTE.value_counts()

DJI                             87042
FIMI                             1463
Hubsan                            837
PARROT                            792
Xiaomi FIMI                       778
                                ...  
LRC                                 1
C Flu fé 2                          1
Cherrocasa                          1
Cherokee                            1
C-Fly Intelligent Technology        1
Name: FABRICANTE, Length: 2707, dtype: int64

Mesmo antes de tratar os nomes dos fabricantes, é possível notar a imensa concentração dos registros de drones com a fabricante DJI.

In [12]:
#Verificação sobre os tipos de uso 
df.TIPO_USO.value_counts()

Básico      103566
Avançado       230
Name: TIPO_USO, dtype: int64

Quase a totalidade dos drones foram registrados como tipo de uso básico.

In [13]:
#Verificação sobre os modelos
df.MODELO.value_counts()

Mavic            21961
MINI 2            4799
Phantom 4 Pro     4648
Phantom 3         4521
Spark             4333
                 ...  
ASW-17               1
SU 35                1
F 16                 1
F 15                 1
Sg700D               1
Name: MODELO, Length: 8898, dtype: int64

Assim como no campo "Fabricante" os principais modelos são da empresa DJI, no entanto, os registros não estão normalizados cada usuário pode ter digitado de uma forma diferente os modelos dos drones.

In [14]:
#Verificação sobre os ramos de atividades
df.RAMO_ATIVIDADE.value_counts()

Recreativo                                                           56765
Aerofotografia                                                       13134
Aerocinematografia                                                   11406
Aeroinspeção                                                          4105
Aerolevantamento - Aerofotogrametria                                  3200
                                                                     ...  
SEGURANÇA AEROPORTUÁRIA                                                  1
Para fins de filmagem e fotos aéreas                                     1
Fotos de terrenos para venda                                             1
Monitoramento Aéreo(Segurança Privada),  Fotos e Filmagens Aéreas        1
Medição Estoque, Complementando a Topografia                             1
Name: RAMO_ATIVIDADE, Length: 3122, dtype: int64

Mais da metade dos registros são de drones destinados para fins recreativos. O campo também trans outras atividades principais com grande representatividade entre os registros.

In [15]:
#Verificação sobre o peso máximo de decolagem informado
df.PESO_MAXIMO_DECOLAGEM_KG.describe()

count    103796.000000
mean         19.166837
std         110.992323
min           0.000000
25%           0.300000
50%           0.750000
75%           1.380000
max        4322.220000
Name: PESO_MAXIMO_DECOLAGEM_KG, dtype: float64

Pela leitura das estatísticas descritivas do campo, a grande maioria dos drones são leves.

In [16]:
#Para agrupar os dados de fabricantes, farei um filtro para encontrar os principais fabricantes e agrupá-los
df[df['FABRICANTE'].str.contains("DJI", na=False)]['FABRICANTE'].value_counts()

DJI                                         87042
DJI MINI SE                                    14
DJI Mavic 2 enterprise zomm                     2
F450 Fabricação Propria com GPS DJI NAza        1
DJI S550 PRO - HEXACOPTERO                      1
PHANTOM 4 RTK/DJI                               1
ELETRONICA DJI FABRICAÇÃO PROPRIA               1
Drone DJI Mavic Mini Mini Drone Dji Mavi        1
SZ DJI TECHNOLOGY CO.                           1
F450 COM MONTAGEM DA DJI                        1
Equipamento montado com sistemas da DJI         1
TBS Clone com controladora DJI Naza             1
SZ DJI TECHINOLOG CO LTD                        1
Fabric própria utilizando Kit DJI F550          1
Name: FABRICANTE, dtype: int64

### Tratamento dos dados de texto

In [17]:
#df['RAMO_ATIVIDADE'] = df['RAMO_ATIVIDADE'].str.casefold()
#Deixa todas as letras minúsculas para realizar a comparação  
df['FABRICANTE'] = df['FABRICANTE'].str.casefold()

In [18]:
#Limpa alguns caracteres especiais
df['FABRICANTE_TRATAMENTO'] = df['FABRICANTE'].str.replace(' ','').replace('/','').replace('-','').replace('.','').replace(',','')

In [19]:
#Verifica todos os nomes de fabricantes que possuem a palavra dji no texto
df[df['FABRICANTE_TRATAMENTO'].str.contains("dji", na=False)]['FABRICANTE_TRATAMENTO'].unique()

array(['dji', '(maker):dji', 'szdjitechnologyco.,ltda',
       'szdjitechnologyco.ltd.', 'djiminise', 'szdjitechnologyco.,ltd',
       'szdjitechnology', 'djiinovationtecnologyco,ldt',
       'szdjitechnologyco.,ltd-6500', 'djispark', 'szdjitechnologyco.ltd',
       'f450fabricaçãopropriacomgpsdjinaza', 'djis550pro-hexacoptero',
       'makerdji',
       'szdjitechnologyco.,ltd14thfloor,westwing,skyworthsemiconductornºdesignbuildingno.18gaoxinsouth4thave,',
       'szdjitechnologyco', 'djimavic2enterprisezomm',
       'szdjitechnologyco.,ltd14thfloor,westwing,skyworthsemiconductornºdesignbuildingno.18gaoxinsouth4thave',
       'djimavic', 'dji-iflighttechnologyco.,ltd.',
       'szdjitechnologycoltd', 'szdjitecnologiaco.',
       'dronedjimavicminifly', 'djitechnology', 'phantom4rtk/dji',
       'szdjitechnologyco.ltda.', 'eletronicadjifabricaçãopropria',
       'djiinvationtechnologyco,ldt', 'dronedjimavicminiminidronedjimavi',
       'szdjitechnologyco.', 'f450commontagemdadji',
    

In [20]:
#Cria uma lista com todos os textos que fazem referência a um drone da dji
dji = ['dji', '(maker):dji', 'szdjitechnologyco.,ltda',
       'szdjitechnologyco.ltd.', 'djiminise', 'szdjitechnologyco.,ltd',
       'szdjitechnology', 'djiinovationtecnologyco,ldt',
       'szdjitechnologyco.,ltd-6500', 'djispark', 'szdjitechnologyco.ltd',
       'djis550pro-hexacoptero',
       'makerdji',
       'szdjitechnologyco.,ltd14thfloor,westwing,skyworthsemiconductornºdesignbuildingno.18gaoxinsouth4thave,',
       'szdjitechnologyco', 'djimavic2enterprisezomm',
       'szdjitechnologyco.,ltd14thfloor,westwing,skyworthsemiconductornºdesignbuildingno.18gaoxinsouth4thave',
       'djimavic', 'dji-iflighttechnologyco.,ltd.',
       'szdjitechnologycoltd', 'szdjitecnologiaco.',
       'dronedjimavicminifly', 'djitechnology', 'phantom4rtk/dji',
       'szdjitechnologyco.ltda.', 'djiinvationtechnologyco,ldt', 'dronedjimavicminiminidronedjimavi',
       'szdjitechnologyco.', 'f450commontagemdadji',
       'szdjitechnologyco.ltd-6500', 'tbsclonecomcontroladoradjinaza', 'szdjitechinologcoltd',
       'szdjitechnologyco.,ltd14thfloor,westwing,skyworthsemiconductorn',
       'szdjitechnologyco,ltda', 'szdjitechnologyco.,ltd6500',
       'szdjitechnologyco,ltd', 'szdjitechnologybco><lyd', 'djimini2fly',
       'szdjitechnologia', 'djicameradrone', 'szdjitecnologycoltd',
       'szdjitecnologyco,ltd', 'szdjitechnologyco,.ltd', "dji'",
       'szdjitechnologyco.,ltd.', 'djii',
       'djiinnovationsscienceandtechnology', 'djimavic2enterprise',
       '(maker)dji', 'djifly', 'szdjitecnologyco.ltd',
       'szdjitechology.co.,ltd.', 'djimini2',
       'djiadà-jiānginnovationsscienceandtechnology', 'ryzetechedji',
       'dji.mini2',
       'szdjitechnologyco.,ltd-650014thfloor,westwing,skyworthsemiconductor,nºdesignbuildingno.18gaoxinsouth4thave,',
       'djitechnologyco.ltd', 'szdjitechnologico',
       'dji-dà-jiānginnovationsscienceandtechnology',
       'shenzhendjisciencesandtechnologiesltd',
       'szdjitechnologycoltd6500', 'ncm/sh/dji',
       'szdjitechnologyco.,ltd14thfloor,westwing,skyworthsemiconductornodesignbuildingno.18gaoxinsouth4thave,',
       'dji-ifiighttechnologyco.,ltd', 'szdjitechnology.co.,ltd', 'dji,2',
       'djiinnovationtechnologyco,ltd', 'szdjitechnologyco.mltd',
       'szdjitechnologyco.,ltd14thfloor,westwing,skyworthsemiconductor,n°designbuildingno.18gaoxinsouth4thave,',
       'dji(dà-jiãnginnovations)', 'djitechnologyco.,ltd.',
       'szdjitechonologyco.,ltd.', 'szdjitechnolyco.,ltd',
       'djimavic2enterprisezoom', 'djitechnologyco.,ltd', 'djimavicpro',
       'djitechnologyco', '\u200edji', 'djipr', 'adji', 'djimas',
       'djiinovationtechnologyco,ldt','mavic', 'mavicair', 'mavic2', 'mavic´pro2', 'dronemavicair2',
       'mavicair2droneprofessionalhd','phanton', 'dronephanton3pro','phantom', 'phantom3se', 'dronephantom4advance','dgimini', 'mini2','dj','djl']

In [21]:
#Cria uma lista com todos os textos que fazem referência a um drone da hubsan
hubsan  = ['hubsan', 'shenzenhubsantecnologia',
       'shenzhenhubsantechnologyco.ltd.', 'dronethehubsan',
       'shenzhenhubsantechnologyco.ltd', 'hubsanzinoh117s',
       'shenzhemhubsantechnologycompanylimit',
       'shenzhenhubsantechnologycoltd.', 'hubsanzino2+plus',
       'hubsanfpvx4brushless', 'hubsanfpvx4brusheess', 'hubsantecnology',
       'shenzhenhubsantechnologycompanylimit', 'hubsan501sbranco',
       'hubsanzinoh117', 'shenzhenhubsan', 'hubsanx4desire', 'hubsan501s',
       'hubsanh501s-s', 'shenzhenhubsantechnologycompanylimi',
       'fabricantehubsan', 'hubsanx4câmerafpv1080psemescovagp',
       'hubsanx4', 'hubsanfpvx4desire', 'hubsanh501ss', 'hubsanzino',
       'hubsanh501ssx4', 'makerhubsanh501ss',
       'hubsantecnologycompanylimited',
       'shenzhenhubsantechnologycompanyltd',
       'shenzhenhubsantechnologycoltd', 'hubsanzinoro', 'hubsanzino117s',
       'hubsanx4desireh502s', 'shenzhenhubsantechnologycompany',
       'hubsanzinoh170s', 'hubsanx4h501s', 'hubsanh501a', 'hubsanzinopro',
       'shenzhenhubsantechnologyco', 'shenzhenhubsantechnologyco.',
       'shenzhenhubsantechnologycompanylimited',
       'shenzhenhubsantechnology', 'shenzhenhubsantechnologyco.,ltd',
       'shenzenhubsantechnologyco.,ltd',
       'shenzhenhubsantechenologyco.,ltd','hubsanzinoh117s', 'hubsanzino2+plus', 'hubsanzinoh117',
       'husbsanzinos117', 'hubsanzino', 'hubsanzinoro', 'hubsanzino117s',
       'hubsanzinoh170s', 'hubsanzinopro']

In [22]:
#Cria uma lista com todos os textos que fazem referência a um drone da xiaomi
fimi = ['fimi', 'xiaomifimi', 'beijingfimitechnologyco.ltd.',
       'beijingfimitechnology', 'beijingfimitechnologylimited',
       'fimix8se', 'fimirobottechnologyco.ltd', 'xiaomifimix8',
       'fimix82020', 'beijingfimitechnologyco.ltd', 'xiaomifimix8se',
       'xioamifimia3', 'shenzhenfimirobottechnologyco.ltd.',
       'shenzhenfimirobottechnologyco.ltd', 'beijingfimitechnology07c',
       'beijingfimitechnologyco.ltda', 'fimitechnologyltda',
       'fimitecnologycoltd', 'fimitechnologyltd',
       'beijingfimitechnologyltd', 'fimitechnologyco.ltd',
       'beijingfimitechnologyelana', 'beijingfimitecchnology',
       'fimi(xiomi)', 'dronefimix8se', 'beijingfimitechnologyco',
       'fimix8se2020', 'fimix8', 'beijingfimitechnologycom.ltd',
       'dronefimix8se2020', 'xiaomi(beijingfimitechnologylimited)',
       'fimisex82020', 'fimimini2', 'fimi/xiaomi',
       'shenzhenfimirobottechnology', 'fimitechnology',
       'shenzhenfimirobottechnologycoltda', 'fimixiaomix8se2020',
       'xiaomi(beijingfimitecnologylimited)', 'xiaomi[fimi]',
       'fimix8sedrone', 'beijinfimitechnology',
       'shenzhenfimirobottechnologyco.,ltd', 'fimi(xiaomi)',
       'beijingfimitechnologyco.ltd(xiaomi)', 'beijingfimi',
       'beijingfimitechinology', 'beijingifimitechnology',
       'beijinfimiinc.ltd', 'xioami(beijingfimitechnologylimted)',
       'dronefimix8se2019', 'beijingfimitechnology07cblockaflo',
       'xiaomifimix8mini', 'dronexiaomifimix8se2020', 'fimise2020',
       'beijingfimitechnologycoltd',
       'beijingfimitechnologylimited-xiaomi',
       'xiaomi-beijingfimitechnologylimited', 'fimi-x8se2020',
       'beijingfimitechnologyco.lyd', 'fimitechnologyltd.',
       'bejingfimitechnology', 'beijingfimitecnologyco',
       'shenzhenfimirobottechnologyco.,ltd.',
       'shenzhenfimirobottechnologyco,ltd', 'beijingfimitechnologuco.',
       'fimirobottechnology', 'beijingfimitechnologyco.,ltd.',
       'xiaomifimix8se2020', 'beijingfimitechnologylimitedfimix8',
       'beijingfimitechnology07c,blocka,floor7,no.28,xinxirodajia,haidiannº',
       'beijingfimitechnologylts', 'beijingfimitecnologyco,ltd',
       'fimitechnologyco.,ltd', 'shenzenfimirobottechnologyco.ltd','xiaomi', 'xiaomifímix8se',
       'xiaomicommunicationstechnologyco.ltd.', 'xiaomimidrone',
       'xiaomitechnologyco.ltd', 'xiaomi.', 'xiaomimi', 'xiaomimidrone4k',
       'xiaomifime', 'xiaomidrone', 'xiaomi-midrine','xiomi','xiome','xioami','xiaome','xiaiomi','xaiomi']

In [23]:
#Cria uma lista com todos os textos que fazem referência a um drone da sjrc
sjrc = ['sjrc', 'sjrcf11', 'sjrcquadcopter', 'sjrcf11gps5gwififpv',
       'sjrcf74kpro', 'sjrcf114kpro', 'fabricante-sjrc', 'dronesjrc',
       'sjrc-f11', 'sjrcf114kpro5gwififpvgps', 'csjrc',
       'sjrcroom38,11/f,meecoindustrialcenternº53-55aupuiwanst.,fotan,n.t.',
       'sjrcf11pró', '*sjrc_drone*', 'sjrcf11s4kpro',
       'sjrcroom34,11/f,meecoindustrialcenternª53-55aupuiwanst.,fotan,n.t.,']

In [24]:
#Cria uma lista com todos os textos que fazem referência a um drone da wld
wld = ['theworldmodels', 'theworldmodels-radar', 'worldmodels',
       'worldmodel', 'theworldmoldel', 'theworldmodel',
       'worldmodelsradar', 'seagull-theworldmodels',
       'aeroworksworldclassaircraft', 'airworld','wordmodel',
        'wordmodels', 'thewordmodels','theworlsmodels']

In [25]:
#Cria uma lista com todos os textos que fazem referência a um drone da iflight
iflight = ['iflight', 'iflight/tbsavionicslimited', 'ifligh',
       'iflightinnovationtechnologyltd', 'iflightnazgulf5dc',
       'iflightchimera5', 'iflightix7', 'cinewhoopiflight', 'iflightgo']

In [26]:
#Cria uma lista com todos os textos que fazem referência a um drone de fabricação própria
artesanal = ['fabricaçãoartesanal', 'fabricaçaõprópria',
       'kitdefabricaçãoprópria', 'fabricaçãoprópria',
       'semfabricante/marcasmescladas', 'fabricaçãopropia',
       'fabricaçãopropria', 'fabricaçãocaseira',
       'f450fabricaçãopropriacomgpsdjinaza', 'fabricantezlldroneportátil',
       'utilizadopeçasdeváriosfabricantes', 'fabricapilot-china',
       'agrosense(fabricaçãoprópria)', 'prototipodefabricaçãopropria',
       'protótipodefabricaçãoprópria', 'fabricantealzrc',
       'fabricantepróprio', 'fabricaçãoprópriaartesanal',
       'fabricantekahakiboy', 'fabricaçaopropria',
       'eletronicadjifabricaçãopropria', 'fabricaçãop´ropria',
       'aeromodelodefabricaçãoprópria', 'aeromodelofabricadoemisoporp3',
       'aeromodelodefabricaçãoartesanal', 'f4ucorsairfabricadoemisopor',
       'aeromodelofabricadoemcasa', 'fabricaçaoprropria',
       'fabricaçãopropria-balsaemonokote', 'fabricacaopropria',
       'fabricantebalsausa', 'fabricaçãoemcnc', 'fabricaçãonacionalcnc',
       'fabricaçãonacionalempastinha', 'fabricantejjrc',
       'droneracingfabricanteeachine', 'aeromodelofabricaçãopropria',
       'fabricaçaopropriaemmadeira', 'fabricacaoartesanal',
       'fabricaçãoartesanal.construçãoprópria.', 'fabricacaocaseira',
       'fabricadoemsãopaulo', 'fabricaçãoartezanal',
       'fabricaçãopropriaconformeplanta',
       'aeromodelofabricantedokitflyingcirc', 'fabricadoemcasa',
       'fabricadopelooperador.', 'f450defabricantenãoseriado',
       'dronecompostopordiversosfabricantes',
       'aviãoaeromodelomodelokestrellfabric', 'kitdofabricantepilotrc',
       'kitdofabricanteaeroworksrc', 'montadocomdiferentesfabricantes',
       'fabricantegenérico', 'kitfabricaçãoprópria',
       'compradode2ªmãosemfabricanteespec', 'semfabricanteespecífico',
       'fabricaçaopropriaaeromodelo', 'fabricaçaopropriaasazagui',
       'utilizapeçasdeváriosfabricantes', 'fabricaçãoprópriacomasafixa',
       'fabricprópriautilizandokitdjif550', 'fabricaçãoparticular',
       'fabricaçaopropia', 'fabricaçãopropriá', 'semnomedefabricante',
       'dronedefabricação-kit', 'fabricaçõcaseira', 'fabricaçãopessoal',
       'owldrones/pixhawk-fabricaçãoprópria', 'fabri', 'nomefabricante','prototipo', 'prototipodefabricaçãopropria', 'protocol'
       ,'fabricaçãoartesanal', 'artesãodiegodiasfortaleza', 'artesanal',
       'artesanalmente', 'fabricaçãoprópriaartesanal',
       'feitoartesanalmenteparapesca', 'aeromodelodefabricaçãoartesanal',
       'aeromodeloartesanal', 'droneartesanalfpv', 'contruçãoartesanal',
       'fabricacaoartesanal', 'montagemartesanal',
       'fabricaçãoartesanal.construçãoprópria.', 'construçãoartesanal',
       'artesanal(caseiro)', 'artesanalfeitoemcasa',
       'artesanalstretaircraft','projetoeconstruçãopróprios', 'aeronaveconstruida',
       'construçãoprópria', 'autoconstruido',
       'construçãoemontagemprópria', 'construçãoartezanal', 'construido',
       'construção', 'construçãopropria.', 'projeto/construçãoprópria',
       'projetoeconstrução:yuriyyatsenko',
       'aeromodelodeconstruçãoprópria', 'projetoeconstruçãopróprio',
       'projetoeconstruçãopróprio.', 'modeloconstruídomanualmente',
       'construçaopropria', 'construçaopropia', 'construidopormimmesmo','ufmsearaterconsultoriaeprojetos',
       'projeto&montagem:fmuavfabiomarcelino',
       'projetoeconstruçãopróprios', 'proj.betoglaser/mont.fabiorenaud',
       'projeto/construçãoprópria', 'projetoeconstrução:yuriyyatsenko',
       'projetoeconstruçãopróprio', 'projetoeconstruçãopróprio.',
       'modelodeprojetoproprio-zenith','equipamentomontadocompeçasvariadas', 'dronemontado', 'rpamontado',
       'dronemontadopormin', 'droneracermontadoehomologadonaanac',
       'droneracermontado', 'montadopelooperador', 'montadopormimmesmo',
       'academiadofpv-montado', 'montadopessoalmente',
       'quadricopteromontadocompeçasdiversas', 'dronemontadopelousuário',
       'aeromodelomontadoemisopor/eletrônica', 'montado', 'montadoemcasa',
       'drodes500montado', 'dronemontadoehomologadoparapesca',
       'dronemontadof450', 'dronecustommontado.',
       'dronemontadocompeçasdiversas.', 'montadopeloproprio',
       'aeromodelomontadokit', 'equipamentomontado', 'kitmontado',
       'dronemontadopormincompeçasdiversas',
       'aeromodeloshockflyermontado', 'aeromodelomontado',
       'droneracermontadoehomologado', 'quadricopteromontado',
       'montadocompeçasindividuais', 'dronemontadomanualmente',
       'dronemontadoehomologadonaanatel', 'dronefreestylemontado2',
       'dronefreestylemontado3', 'equipamentomontadocomsistemasdadji',
       'droneracermontado4polegadas', 'dronemontadocompeçaschinesas',
       'dronemontadoehomologadoanatel', 'montadopersonalizado',
       'supertucanomontadoemisopor', 'dronemontadocompeçasvariadas',
       'droneracermontadoehologadonaanatel', 'droneracer(montado)',
       'montadocompeçasdiversas', 'montadopessoalmente-hobby',
       'dronemontadocomcâmera', 'montadomanualmente',
       'montadopelopropriet´rio', 'montadopeloproprietário']

In [27]:
#Cria uma lista com todos os textos que fazem referência a um drone da shantou
shantou = ['shantoukaishengfengtoysco.,ltd.',
       'shantouzhongliintelligenttechnologyc0.,ltd.',
       'shantoukaishengfengtoys',
       'shantouzhongliintelligenttechnologyco.,ltd808th,westfengxin1stroad,chenghainºchina',
       'shantoukaishengfengtoysco', 'shantoucityxinlin',
       'shantouzhongliintelligenttechnologyco.,ltd',
       'shantoukaisheengfengtoysco.ltd', 'shantoukaishengfengtoysco.ltd',
       'shantouxiaowanguotradingco.ltd.',
       'shantoucityxinlintoysindustrialco.',
       'shantouxinlintoysindustrialcompany',
       'shantouzhongliintelligenttechnology',
       'shantoucityxinlintpysco.ltd', 'kf102-shantoukaishengfengtoysco.',
       'shantouzhongliintelligenttechnologyco',
       'shantoucityxinlintoysindu', 'shantouxiaowangguotradingco,ltd.',
       'shantouzhongliinteligenttechnologyco',
       'shantoukaishengfengtoysco.,ltd',
       'shantouzhongliintelligenttechnologyco.',
       'shantoucityxinlintoysindustrialco.,ltd.',
       'shantoukaishengfengtoysco.ldt.',
       'shantouxiaowangguotradingco.ltd.',
       'shantoukaishengfengtoysco.ltd.', 'shantoukaishengfeng',
       'shantouzhongliintelligenttechnologyc',
       'shantoukaishengfengtoyscoltda',
       'shantouzhongliintelligenttechnologyco.ltd.',
       'shantoucityxinlintoysindustrialco', 'shantouchenghai',
       'shantoukaishengfengtoysco.ltda.',
       'shantoucityxinlintoysindustrialco.,ltd', 'shantou',
       'shantoucytixinlintoysindustrialc',
       'shantoucityxintoysindustrialco.ltd',
       'shantoucityxinlintoysindustrial',
       'shantoucitylintoysindustrialco.ltd',
       'shantouzhongliintelligenttech.co.lt',
       'shantoukaishengfengtoyscoltd',
       'shantoucityxinlintoysindustrialltd',
       'shantoucitychenghaiareahuadatoyco.',
       'shantoumeijiaxinplastictoysco.ltd',
       'shantourunqiaplastictoysfactory',
       'shantouyearlotoysco.,ltd.southofthemiddlesectionoffengxiang',
       'shantoukaishengfengtoysco.,ltd.fengxiangstreet',
       'shantouzhongilintelligenttechnologyco,ltda',
       'shantoucityxinlintoysindustrialco.ltd',
       'kfplan(shantoukaishengfengtoysco.,ltd.)',
       'shantouxinlintoysindustrialco.ltd',
       'shantoucityxinlinbrinquedosindustrialco.,ltd.',
       'shantoukaishengfengtoyscoltd.',
       'shantouzhongliintelligenttechnologyco.ltd',
       'kfplan(shantoukaisheengfengtoysco.ltd)',
       'shantouchuangqiinteligenttechnologyco.ltda.',
       'shantoucityxinlintoys', 'shantoucityxinlintoysindustrialco.lto',
       'shantou/\t\u200ekfplan', 'shantoucityxinlintoysco.,ltd(zll)',
       'shantoucityxinlintoysindustrialco.,ltdn°.6xinjinroadzhujinindustrialzonenºlonghudistrict,shantoucity',
       'shantoukaishenfengtoysco.,ltd.',
       'shantouchenghaikaishengfengtoysco.,ltd',
       'shantoukaishengfengtoysco.,ltd.fengxiangstreetnºgangkousection,fengxin1stroad,chengaidist.,',
       'shantouxiaowanguotradingcoltda', 'shantouk.toysco.ltd',
       'shantoukaishengfngtoysco.,ltd.', 'shantoucitytoys',
       'shantoukaishengfengtoysco,ltd', 'shantoukaishengfengtoysco,ltd,',
       'shantoukaishengfengtoys.ltd', 'shantouzhongliintelligent',
       'shantouzhongliintelligenttechnologyco.,ltda.', 'shantoucity',
       'shantouksftoysco.,ltd', 'shantoucityxinlintoyscoltd',
       'shantouchenghaidistrictlianxiafuqifábricadeprocessamentodeplástico',
       'shantouzhongliintelligenttecnology']

In [28]:
#Para facilitar a identificação dos drones das principais marcas, filtramos os drones de cada fabricante com base nos textos encontrados com o nome do fabricante e renomeamos por um nome padronizado 
df.FABRICANTE_TRATAMENTO.replace(dji, 'DJI', inplace=True)
df.FABRICANTE_TRATAMENTO.replace(fimi, 'XIAOMI', inplace=True)
df.FABRICANTE_TRATAMENTO.replace(hubsan, 'HUBSAN', inplace=True)
df.FABRICANTE_TRATAMENTO.replace(sjrc, 'SJRC', inplace=True)
df.FABRICANTE_TRATAMENTO.replace(wld, 'WORLD MODEL', inplace=True)
df.FABRICANTE_TRATAMENTO.replace(iflight, 'IFLIGHT', inplace=True)
df.FABRICANTE_TRATAMENTO.replace(artesanal, 'ARTESANAL', inplace=True)
df.FABRICANTE_TRATAMENTO.replace(shantou, 'SHANTOU', inplace=True)


In [29]:
#Abaixo é realizada a padronização do tipo de uso do drone com base na identificação do campo código aeronave
#Essa identificação encontra-se no link: https://www.anac.gov.br/acesso-a-informacao/dados-abertos/areas-de-atuacao/aeronaves/drones-cadastrados/9-drones-cadastrados
uso = {'PR':'Recreativo (Aeromodelo)'
      ,'PP':'Não recreativo básico'
      ,'PS':'Uso avançado'}

df['USO'] = df['CODIGO_AERONAVE'].str.slice(0,2).map(uso)

In [30]:
#verifica os valores distintos e a contagem no campo uso
df['USO'].value_counts()

Recreativo (Aeromodelo)    56764
Não recreativo básico      46802
Uso avançado                 230
Name: USO, dtype: int64

In [31]:
#Exibe as primeiras e últimas linhas do DataFrame
df

,CODIGO_AERONAVE,DATA_VALIDADE,TIPO_USO,FABRICANTE,MODELO,PESO_MAXIMO_DECOLAGEM_KG,RAMO_ATIVIDADE,FABRICANTE_TRATAMENTO,USO
0,PP- 11082012,05/01/2024,Básico,dji,Mavic,0.743,Aerocinematografia,DJI,Não recreativo básico
1,PP- 26534803,07/04/2024,Básico,dji,Mavic 2 Pro,9.000,Aerofotografia,DJI,Não recreativo básico
2,PP- 35041299,03/01/2024,Básico,dji,Mavic,0.128,Aeroinspeção,DJI,Não recreativo básico
3,PP- 40211550,05/05/2023,Básico,dji,Phantom 4 Pro,1.388,Aerolevantamento - Aerofotogrametria,DJI,Não recreativo básico
4,PP-000000001,26/03/2023,Básico,dji,Mavic,1.000,Aerocinematografia,DJI,Não recreativo básico
...,...,...,...,...,...,...,...,...,...
103807,PR-548746952,05/10/2024,Básico,dji,Mini,0.250,Recreativo,DJI,Recreativo (Aeromodelo)
103808,PR-181298149,20/10/2024,Básico,c-fly intelligent technology,Faith 2S,0.570,Recreativo,c-flyintelligenttechnology,Recreativo (Aeromodelo)
103809,PR-139126124,07/10/2024,Básico,zll,Sg700D,612.600,Recreativo,zll,Recreativo (Aeromodelo)
103810,PR-281220000,12/10/2024,Básico,dji,Phantom 2,1.000,Recreativo,DJI,Recreativo (Aeromodelo)


In [32]:
#Substitui as informações originais da coluna fabricante pelos dados normalizados da coluna fabricante_tratamento
#Apaga os campos que não serão utilizados na análise
df['FABRICANTE'] = df['FABRICANTE_TRATAMENTO']
del df['FABRICANTE_TRATAMENTO']
del df['RAMO_ATIVIDADE']
del df['CODIGO_AERONAVE']
del df['DATA_VALIDADE']
del df['PESO_MAXIMO_DECOLAGEM_KG']


In [33]:
#Remove o "_" presente nos nomes dos campos
df.columns = df.columns.str.replace('_',' ')

In [34]:
#Grava o resultado final em um arquivo ".csv"
df.to_csv('SISANT_20221026.csv', sep='|',index=False, float_format='%.3f')